### Overview
We will provide the detailed reproducing steps and results. For prviacy and security reason, output of some intermediate configuring and compiling processes are cleared (which may leak our file directories). However, you should execute ALL scripts sequentially. If you only want to execute part of the scripts, take note of the dependencies of the scripts shown as follows:

- Setup: dependency of Figures 8, 9, 10, 11
- Figure 8: dependency of Figures 9, 10, 11

For example, if you want to draw either Figure 9, 10, 11, you need to at least have executed Setup and Figure 8 scripts.

The detailed documentation of each specific experiment can also be found in [GFE Driver for RadixGraph](https://github.com/ForwardStar/gfe_driver).

### Minimum requirement

- OS: Linux (ubuntu is recommended);

- Cores: at least supporting 64 threads;

- Memory: >= 256G.

### Setup

Prepare [GFE Driver for RadixGraph](https://github.com/ForwardStar/gfe_driver) and download datasets:

In [1]:
%%bash
git clone https://github.com/ForwardStar/gfe_driver.git
cd gfe_driver
sh scripts/prepare_datasets.sh

Cloning into 'gfe_driver'...


Fetching from https://zenodo.org/records/3966439/files/datasets.tar.gz...


100%|██████████| 23.2G/23.2G [2:27:56<00:00, 2.81MB/s]  


Fetching from https://snap.stanford.edu/data/bigdata/communities/com-lj.ungraph.txt.gz...


100%|██████████| 119M/119M [00:22<00:00, 5.60MB/s]   


...

Fetching from https://snap.stanford.edu/data/bigdata/communities/com-friendster.ungraph.txt.gz...


100%|██████████| 8.73G/8.73G [14:42<00:00, 10.6MB/s]   


Fetching from https://snap.stanford.edu/data/twitter-2010.txt.gz...


100%|██████████| 5.12G/5.12G [08:56<00:00, 10.3MB/s]   


Extracting datasets... done


Note 1: the prepare_datasets script requires Python and some packages. Install manually if you haven't done so.

Note 2: downloading data from zenodo is fairly slow, but it may not be the downloader or your network's fault. It is just because zenodo server is slow. Please be patient. If the speed is extremely slow (e.g., 100kb/s), try re-running the script several times until it converges to a normal speed.

Now install the dependencies for GFE Driver if you haven't:

In [ ]:
!sudo apt install gcc-11 g++-11 libnuma-dev cmake zlib1g-dev libpapi-dev libjemalloc-dev python3 libtbb-dev libevent-dev libboost-all-dev

Configure the GFE Driver:

In [ ]:
%%bash
cd gfe_driver
git submodule update --init
mkdir build
cd build
autoreconf -iv ..

### Insertions, Deletions and Memory Consumptions (Figure 8)

#### Generate vertex operations

In [3]:
%%bash
cd gfe_driver
g++ scripts/create_vertex_ops.cpp -o create_vertex_ops -O3
./create_vertex_ops

#### RadixGraph:

(1) Configure and compile:

In [ ]:
%%bash
cd gfe_driver/library/radixgraph/RadixGraph
git submodule update --init --recursive
cmake -S . -DCMAKE_BUILD_TYPE=Release
make -j
g++ optimizer.cpp -o optimizer -O3
mv optimizer ../../../
cd ../../../build
../configure --enable-optimize --disable-debug --enable-mem-analysis --with-radixgraph=../library/radixgraph/RadixGraph
make clean && make -j

(2) Run:

In [5]:
%%bash
cd gfe_driver
sh scripts/run_random.sh radixgraph 64
sh scripts/run_vertex.sh radixgraph 64

Reading vertex IDs from datasets/graph500-24.v ...
Number of distinct IDs: 8870942
Bit length of IDs: 24
Optimal fanout settings saved to settings.txt
Format: <number_of_layers>\n<fanout_array>
Reading vertex IDs from datasets/uniform-24.v ...
Number of distinct IDs: 8870942
Bit length of IDs: 24
Optimal fanout settings saved to settings.txt
Format: <number_of_layers>\n<fanout_array>
Reading vertex IDs from datasets/dota-league.v ...
Number of distinct IDs: 61170
Bit length of IDs: 19
Optimal fanout settings saved to settings.txt
Format: <number_of_layers>\n<fanout_array>
Reading vertex IDs from datasets/com-lj.ungraph.el ...
Number of distinct IDs: 3997962
Bit length of IDs: 22
Optimal fanout settings saved to settings.txt
Format: <number_of_layers>\n<fanout_array>
Reading vertex IDs from datasets/com-orkut.ungraph.el ...
Number of distinct IDs: 3072441
Bit length of IDs: 22
Optimal fanout settings saved to settings.txt
Format: <number_of_layers>\n<fanout_array>
Reading vertex IDs fro

#### Teseo:

(1) Configure and compile:

In [ ]:
%%bash
cd gfe_driver
git clone https://github.com/cwida/teseo
cd teseo
autoreconf -iv
mkdir build
cd build
../configure --enable-optimize --disable-debug
make -j
cd ../../build
../configure --enable-optimize --disable-debug --enable-mem-analysis --with-teseo=../teseo/build
make clean && make -j

(2) Run:

In [7]:
%%bash
cd gfe_driver
sh scripts/run_random.sh teseo.13 64
sh scripts/run_vertex.sh teseo.13 64

Killed
Killed


CalledProcessError: Command 'b'cd gfe_driver\nsh scripts/run_random.sh teseo.13 64\nsh scripts/run_vertex.sh teseo.13 64\n'' returned non-zero exit status 137.

You may notice that for edge operations on Twitter dataset, Teseo fails to complete within 48 hours. If so, manually kill the process (for example, by htop). Also, you may observe a deadlock when Teseo finishes vertex operations on Twitter dataset. These processes for Twitter have following commands.

#### Sortledton:

(1) Configure and compile:

In [ ]:
%%bash
cd gfe_driver
git clone https://gitlab.db.in.tum.de/per.fuchs/sortledton
cd sortledton
git reset --hard
sed -i '11d' CMakeLists.txt
mkdir build
cd build
cmake -DCMAKE_BUILD_TYPE=Release ..
make sortledton
cd ../../build
../configure --enable-optimize --disable-debug --enable-mem-analysis --with-sortledton=../sortledton/build/
make clean && make -j

(2) Run:

In [9]:
%%bash
cd gfe_driver
sh scripts/run_random.sh sortledton.4 64
sh scripts/run_vertex.sh sortledton.4 64

terminate called after throwing an instance of 'EdgeExistsException'
Aborted (core dumped)


You may notice that Sortledton will raise an exception on Twitter dataset.

#### Spruce

(1) Configure and compile:

In [ ]:
%%bash
cd gfe_driver
wget -c https://github.com/Stardust-SJF/gfe_driver/releases/download/v2.0.0/libBVGT_stable.a
mv libBVGT_stable.a libBVGT.a
cd build
../configure --enable-optimize --enable-mem-analysis --disable-debug --with-bvgt=../

Install [junction](https://github.com/preshing/junction) dependency for Spruce.

In [ ]:
%%bash
cd gfe_driver
git clone https://github.com/preshing/junction.git
git clone https://github.com/preshing/turf.git
cd junction
mkdir build
cd build
cmake -DCMAKE_INSTALL_PREFIX=~/junction-install -DJUNCTION_WITH_SAMPLES=OFF ..
make install

Manually configure junction's include and lib paths to ``gfe_driver/build/Makefile``. The junction installation folder should be ``~/junction-install`` if you follow the above commands. Refer to [https://github.com/ForwardStar/gfe_driver?tab=readme-ov-file#spruce](https://github.com/ForwardStar/gfe_driver?tab=readme-ov-file#spruce) for details. Then compile with following commands.

In [ ]:
%%bash
cd gfe_driver/build
make clean && make -j

(2) Run:

In [13]:
%%bash
cd gfe_driver
sh scripts/run_random.sh bvgt 64
sh scripts/run_vertex.sh bvgt 64

Finally, save the Makefile for future use.

In [3]:
%%bash
cd gfe_driver
mv build/Makefile build/Makefile_spruce

#### GTX

(1) Configure and compile:

In [ ]:
%%bash
cd gfe_driver
git clone https://github.com/Jiboxiake/GTX-SIGMOD2025
cd GTX-SIGMOD2025
git submodule update --init --recursive
mkdir build
cd build
cmake -DCMAKE_BUILD_TYPE=Release ..
make -j
cd ../../build
../configure --enable-optimize --disable-debug --enable-mem-analysis --with-gtx=../GTX-SIGMOD2025/build
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:../GTX-SIGMOD2025/build
make clean && make -j

If you failed to compile GTX, refer to [https://github.com/ForwardStar/gfe_driver?tab=readme-ov-file#gtx-1](https://github.com/ForwardStar/gfe_driver?tab=readme-ov-file#gtx-1).

(2) Run:

In [10]:
%%bash
cd gfe_driver
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:./GTX-SIGMOD2025/build
sh scripts/run_random.sh gtx 64
sh scripts/run_vertex.sh gtx 64

Killed


You may notice that for edge operations on Twitter dataset, GTX fails to complete within 48 hours. If so, manually kill the process (for example, by htop). The process has following command.

#### Generate Figure 8

In [1]:
%%bash
cd gfe_driver
python3 scripts/data_to_csv_components/edge_operations.py
python3 scripts/data_to_csv_components/memory_consumption.py
python3 scripts/data_to_csv_components/vertex_operations.py
python3 scripts/data_to_csv_components/vertex_memory.py
python3 scripts/plot_all_1.py

Writing update throughput results to CSV...
Done.
Writing memory results to CSV...
Done.
Processing gtx
Processing file graph500-24-vertices-gtx
Processing file com-lj.ungraph-vertices-gtx
Processing file dota-league-vertices-gtx
Processing file com-orkut.ungraph-vertices-gtx
Processing file uniform-24-vertices-gtx
Processing file twitter-2010-vertices-gtx
Processing teseo.13
Processing file com-orkut.ungraph-vertices-teseo.13
Processing file dota-league-vertices-teseo.13
Processing file com-lj.ungraph-vertices-teseo.13
Processing file uniform-24-vertices-teseo.13
Processing file graph500-24-vertices-teseo.13
Processing file twitter-2010-vertices-teseo.13
Processing radixgraph
Processing file com-orkut.ungraph-vertices-radixgraph
Processing file com-lj.ungraph-vertices-radixgraph
Processing file twitter-2010-vertices-radixgraph
Processing file uniform-24-vertices-radixgraph
Processing file graph500-24-vertices-radixgraph
Processing file dota-league-vertices-radixgraph
Processing bvgt
P

In [2]:
from IPython.display import IFrame, display

display(IFrame("gfe_driver/figures/combined_plots1.pdf", width=800, height=300))

If you reach here successfully, congrats! The rest procedures will be much easier, as you have completed the most difficult part (configure and compile each system).

### Mixed updates and delete-only (Figure 9)

#### Generate graphlog

(1) Compile:

In [ ]:
%%bash
cd gfe_driver
git clone https://github.com/MordorsElite/graphlog-base.git
cd graphlog-base
git submodule update --init
mkdir build
cd build
cmake ../ -DCMAKE_BUILD_TYPE=Release
make -j

Note: the original graphlog repo is [https://github.com/whatsthecraic/graphlog](https://github.com/whatsthecraic/graphlog), but it only works on GCC 10 and pops build errors for GCC 11+. So here we use one of its forks which fixes the issue.

(2) Generate:

In [2]:
%%bash
cd gfe_driver/graphlog-base/build
./graphlog -a 1 -e 1 -v 1 ../../datasets/graph500-24.properties ../../graph500-24-delete.graphlog
./graphlog -a 1 -e 1 -v 1 ../../datasets/uniform-24.properties ../../uniform-24-delete.graphlog
./graphlog -a 10 -e 1 -v 1 ../../datasets/graph500-24.properties ../../graph500-24-1.0.graphlog
./graphlog -a 10 -e 1 -v 1 ../../datasets/uniform-24.properties ../../uniform-24-1.0.graphlog

Path input graph: ../../datasets/graph500-24.properties
Path output log: ../../graph500-24-delete.graphlog
Aging factor: 1
Expansion factor for the vertices: 1
Expansion factor for the edges: 1
Seed for the random generator: 437576195

Reading the input graph from: ../../datasets/graph500-24.properties ... 
The final graph will contain 8870942 vertices and 260379520 edges
Input graph parsed in 1:33 mins
Generating 0 (0 %) non final vertices ... 
Vertices generated in 220 ms
Initialising the counting tree for 8870942 vertices ... 
Counting tree created in 278 ms
Permuting the edges in the final graph ... 
Permutation completed in 8.850 s
Initialising the log file ....
Compressing and saving 8870942 vertices ...
List of vertices serialised in 71.994 s
Compressing and saving 0 vertices ...
List of vertices serialised in 13 us
Log file initialised in 71.994 s
Generating 260379520 operations ...
Progress: 2603796/260379520 (1 %), edges final: 2603796/260379520 (1 %), edges temp: 0/2603796 (

#### RadixGraph

(1) Configure and compile:

In [ ]:
%%bash
cd gfe_driver/build
../configure --enable-optimize --disable-debug --enable-mem-analysis --with-radixgraph=../library/radixgraph/RadixGraph
make clean && make -j

(2) Run:

In [3]:
%%bash
cd gfe_driver
sh scripts/run_mixed.sh radixgraph 64
sh scripts/run_delete_memfp.sh radixgraph 2

Reading vertex IDs from datasets/graph500-24.v ...
Number of distinct IDs: 8870942
Bit length of IDs: 24
Optimal fanout settings saved to settings.txt
Format: <number_of_layers>\n<fanout_array>
Reading vertex IDs from datasets/uniform-24.v ...
Number of distinct IDs: 8870942
Bit length of IDs: 24
Optimal fanout settings saved to settings.txt
Format: <number_of_layers>\n<fanout_array>
Reading vertex IDs from datasets/graph500-24.v ...
Number of distinct IDs: 8870942
Bit length of IDs: 24
Optimal fanout settings saved to settings.txt
Format: <number_of_layers>\n<fanout_array>
Reading vertex IDs from datasets/uniform-24.v ...
Number of distinct IDs: 8870942
Bit length of IDs: 24
Optimal fanout settings saved to settings.txt
Format: <number_of_layers>\n<fanout_array>


#### Teseo

(1) Configure and compile:

In [ ]:
%%bash
cd gfe_driver/build
../configure --enable-optimize --disable-debug --enable-mem-analysis --with-teseo=../teseo/build
make clean && make -j

(2) Run:

In [ ]:
%%bash
cd gfe_driver
sh scripts/run_mixed.sh teseo.13 64
sh scripts/run_delete_memfp.sh teseo.13 2

#### Sortledton

(1) Configure and compile:

In [ ]:
%%bash
cd gfe_driver/build
../configure --enable-optimize --disable-debug --enable-mem-analysis --with-sortledton=../sortledton/build/
make clean && make -j

(2) Run:

In [2]:
%%bash
cd gfe_driver
sh scripts/run_mixed.sh sortledton.4 64
sh scripts/run_delete_memfp.sh sortledton.4 2

#### Spruce

(1) Configure and compile:

In [ ]:
%%bash
cd gfe_driver/build
mv Makefile_spruce Makefile
make clean && make -j

(2) Run:

In [7]:
%%bash
cd gfe_driver
sh scripts/run_mixed.sh bvgt 64
sh scripts/run_delete_memfp.sh bvgt 2
mv build/Makefile build/Makefile_spruce

#### GTX

(1) Configure and compile:

In [ ]:
%%bash
cd gfe_driver/build
../configure --enable-optimize --disable-debug --enable-mem-analysis --with-gtx=../GTX-SIGMOD2025/build
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:../GTX-SIGMOD2025/build
make clean && make -j

(2) Run:

In [9]:
%%bash
cd gfe_driver
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:./GTX-SIGMOD2025/build
sh scripts/run_mixed.sh gtx 64
sh scripts/run_delete_memfp.sh gtx 2

#### Generate Figure 9

In [11]:
%%bash
cd gfe_driver
python3 scripts/plot_all_2.py

Processing gtx
Processing file uniform-24-mixed-gtx.sqlite3
Processing file graph500-24-mixed-gtx.sqlite3
Processing teseo.13
Processing file graph500-24-mixed-teseo.13.sqlite3
Processing file uniform-24-mixed-teseo.13.sqlite3
Processing radixgraph
Processing file graph500-24-mixed-radixgraph.sqlite3
Processing file uniform-24-mixed-radixgraph.sqlite3
Processing bvgt
Processing file graph500-24-mixed-bvgt.sqlite3
Processing file uniform-24-mixed-bvgt.sqlite3
Processing sortledton.4
Processing file graph500-24-mixed-sortledton.4.sqlite3
Processing file uniform-24-mixed-sortledton.4.sqlite3
Processing gtx
Processing file graph500-24-mixed-gtx
Processing file uniform-24-mixed-gtx
Processing teseo.13
Processing file uniform-24-mixed-teseo.13
Processing file graph500-24-mixed-teseo.13
Processing radixgraph
Processing file graph500-24-mixed-radixgraph
Processing file uniform-24-mixed-radixgraph
Processing bvgt
Processing file graph500-24-mixed-bvgt
Processing file uniform-24-mixed-bvgt
Proce

In [16]:
from IPython.display import IFrame, display

display(IFrame("gfe_driver/figures/combined_plots2.pdf", width=700, height=200))

### Graph analytics (Figure 10)

#### Generate property files for datasets

In [ ]:
%%bash
cd gfe_driver
g++ generate_property_files.cpp -o generate_property_files -O3
./generate_property_files datasets/twitter-2010.el
./generate_property_files datasets/com-lj.ungraph.el
./generate_property_files datasets/com-orkut.ungraph.el

#### Enable algorithm BC

Add following lines to to ``gfe_driver/datasets/dota-league.properties``, ``gfe_driver/datasets/graph500-24.properties`` and ``gfe_driver/datasets/uniform24.properties``:

#### RadixGraph

(1) Configure and compile:

In [ ]:
%%bash
cd gfe_driver/build
../configure --enable-optimize --disable-debug --enable-mem-analysis --with-radixgraph=../library/radixgraph/RadixGraph
make clean && make -j

(2) Run:

In [ ]:
%%bash
cd gfe_driver
sh scripts/run_analytics.sh radixgraph 64

#### Teseo

(1) Configure and compile:

In [ ]:
%%bash
cd gfe_driver/build
../configure --enable-optimize --disable-debug --enable-mem-analysis --with-teseo=../teseo/build
make clean && make -j

(2) Run:

In [ ]:
%%bash
cd gfe_driver
sh scripts/run_analytics.sh teseo.13 64

#### Sortledton

(1) Configure and compile:

In [ ]:
%%bash
cd gfe_driver/build
../configure --enable-optimize --disable-debug --enable-mem-analysis --with-sortledton=../sortledton/build
make clean && make -j

(2) Run:

In [ ]:
%%bash
cd gfe_driver
sh scripts/run_analytics.sh sortledton.4 64

#### Spruce

(1) Configure and compile:

In [ ]:
%%bash
cd gfe_driver/build
mv Makefile_spruce Makefile
make clean && make -j

(2) Run:

In [ ]:
%%bash
cd gfe_driver
sh scripts/run_analytics.sh bvgt 64
mv build/Makefile build/Makefile_spruce

#### GTX

(1) Configure and compile:

In [ ]:
%%bash
cd gfe_driver/build
../configure --enable-optimize --disable-debug --enable-mem-analysis --with-gtx=../GTX-SIGMOD2025/build
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:../GTX-SIGMOD2025/build
make clean && make -j

(2) Run:

In [ ]:
%%bash
cd gfe_driver
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:./GTX-SIGMOD2025/build
sh scripts/run_analytics.sh gtx 64

#### Generate Figure 10

In [ ]:
%%bash
cd gfe_driver
python3 scripts/data_to_csv_components/graph_analytics.py
python3 scripts/plot_all_3.py

In [ ]:
from IPython.display import IFrame, display

display(IFrame("gfe_driver/figures/combined_plots3.pdf", width=800, height=300))

### Concurrent reads and writes (Figure 11)

#### Generate graphlog

In [3]:
%%bash
cd gfe_driver/graphlog-base/build
./graphlog -a 10 -e 1 -v 1 ../../datasets/dota-league.properties ../../dota-league.graphlog

Path input graph: ../../datasets/dota-league.properties
Path output log: ../../dota-league.graphlog
Aging factor: 10
Expansion factor for the vertices: 1
Expansion factor for the edges: 1
Seed for the random generator: 1774627485

Reading the input graph from: ../../datasets/dota-league.properties ... 
The final graph will contain 61170 vertices and 50870313 edges
Input graph parsed in 9.519 s
Generating 0 (0 %) non final vertices ... 
Vertices generated in 943 us
Initialising the counting tree for 61170 vertices ... 
Counting tree created in 2 ms
Permuting the edges in the final graph ... 
Permutation completed in 2.196 s
Initialising the log file ....
Compressing and saving 61170 vertices ...
List of vertices serialised in 562 ms
Compressing and saving 0 vertices ...
List of vertices serialised in 24 us
Log file initialised in 563 ms
Generating 508703130 operations ...
Progress: 5087032/508703130 (1 %), edges final: 508704/50870313 (1 %), edges temp: 4569210/5077914 (89.982 %), ht si

#### RadixGraph

(1) Enable 1-hop neighbor queries in ``gfe_driver/configuration.hpp``:

In [1]:
%%bash
sed -i \
  -e 's/^#define[[:space:]]\+RUN_GET_NEIGHBORS[[:space:]]\+[01]/#define RUN_GET_NEIGHBORS 1/' \
  -e 's/^#define[[:space:]]\+RUN_TWO_HOP_NEIGHBORS[[:space:]]\+[01]/#define RUN_TWO_HOP_NEIGHBORS 0/' \
  gfe_driver/configuration.hpp

(2) Configure and compile:

In [ ]:
%%bash
cd gfe_driver/build
../configure --enable-optimize --disable-debug --enable-mem-analysis --with-radixgraph=../library/radixgraph/RadixGraph
make clean && make -j

(3) Run and save the results:

In [3]:
%%bash
cd gfe_driver
sh scripts/run_concurrent.sh radixgraph
mv results/radixgraph/concurrent results/radixgraph/concurrent-1-hop

Reading vertex IDs from datasets/dota-league.v ...
Number of distinct IDs: 61170
Bit length of IDs: 19
Optimal fanout settings saved to settings.txt
Format: <number_of_layers>\n<fanout_array>


(4) Enable 2-hop neighbor queries in ``gfe_driver/configuration.hpp``:

In [6]:
%%bash
sed -i \
  -e 's/^#define[[:space:]]\+RUN_GET_NEIGHBORS[[:space:]]\+[01]/#define RUN_GET_NEIGHBORS 0/' \
  -e 's/^#define[[:space:]]\+RUN_TWO_HOP_NEIGHBORS[[:space:]]\+[01]/#define RUN_TWO_HOP_NEIGHBORS 1/' \
  gfe_driver/configuration.hpp

(5) Re-compile:

In [ ]:
%%bash
cd gfe_driver/build
make -j

(6) Re-run and save the results:

In [9]:
%%bash
cd gfe_driver
sh scripts/run_concurrent.sh radixgraph
mv results/radixgraph/concurrent results/radixgraph/concurrent-2-hop

Reading vertex IDs from datasets/dota-league.v ...
Number of distinct IDs: 61170
Bit length of IDs: 19
Optimal fanout settings saved to settings.txt
Format: <number_of_layers>\n<fanout_array>


#### GTX

(1) Enable 1-hop neighbor queries in ``gfe_driver/configuration.hpp``:

In [10]:
%%bash
sed -i \
  -e 's/^#define[[:space:]]\+RUN_GET_NEIGHBORS[[:space:]]\+[01]/#define RUN_GET_NEIGHBORS 1/' \
  -e 's/^#define[[:space:]]\+RUN_TWO_HOP_NEIGHBORS[[:space:]]\+[01]/#define RUN_TWO_HOP_NEIGHBORS 0/' \
  gfe_driver/configuration.hpp

(2) Configure and compile:

In [ ]:
%%bash
cd gfe_driver/build
../configure --enable-optimize --disable-debug --enable-mem-analysis --with-gtx=../GTX-SIGMOD2025/build
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:../GTX-SIGMOD2025/build
make clean && make -j

(3) Run and save the results:

In [12]:
%%bash
cd gfe_driver
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:./GTX-SIGMOD2025/build
sh scripts/run_concurrent.sh gtx
mv results/gtx/concurrent results/gtx/concurrent-1-hop

(4) Enable 2-hop neighbor queries in ``gfe_driver/configuration.hpp``:

In [13]:
%%bash
sed -i \
  -e 's/^#define[[:space:]]\+RUN_GET_NEIGHBORS[[:space:]]\+[01]/#define RUN_GET_NEIGHBORS 0/' \
  -e 's/^#define[[:space:]]\+RUN_TWO_HOP_NEIGHBORS[[:space:]]\+[01]/#define RUN_TWO_HOP_NEIGHBORS 1/' \
  gfe_driver/configuration.hpp

(5) Re-compile:

In [ ]:
%%bash
cd gfe_driver/build
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:../GTX-SIGMOD2025/build
make -j

(6) Re-run and save the results:

In [1]:
%%bash
cd gfe_driver
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:./GTX-SIGMOD2025/build
sh scripts/run_concurrent.sh gtx
mv results/gtx/concurrent results/gtx/concurrent-2-hop

#### Generate Figure 11

In [2]:
%%bash
cd gfe_driver
printf "concurrent-1-hop\nconcurrent-2-hop\n" > concurrent_results.txt
python3 scripts/plot_concurrent.py < ./concurrent_results.txt

Please enter the path to the concurrent one-hop neighbor results.
E.g., if the path is 'results/radixgraph/concurrent-1-hop', just enter 'concurrent-1-hop'.
Enter the folder name for one-hop neighbor results: Please enter the path to the concurrent two-hop neighbor results.
E.g., if the path is 'results/radixgraph/concurrent-2-hop', just enter 'concurrent-2-hop'.
Enter the folder name for two-hop neighbor results: Processing gtx
Processing file dota-league-16-write-threads-gtx
Processing file dota-league-8-read-threads-gtx
Processing file dota-league-32-write-threads-gtx
Processing file dota-league-8-write-threads-gtx
Processing file dota-league-4-read-threads-gtx
Processing file dota-league-32-read-threads-gtx
Processing file dota-league-16-read-threads-gtx
Processing file dota-league-4-write-threads-gtx
Processing teseo.13
Processing radixgraph
Processing file dota-league-4-read-threads-radixgraph
Processing file dota-league-8-read-threads-radixgraph
Processing file dota-league-16-re

In [4]:
from IPython.display import IFrame, display

display(IFrame("gfe_driver/figures/combined_plots4.pdf", width=600, height=200))

### Case study of SORT (Figures 12, 13)

#### Compile RadixGraph and test modules

In [ ]:
%%bash
g++ optimizer.cpp -o optimizer -O3
mkdir build
cd build
cmake ..
make -j

#### Figure 12

Figure 12(b) is generated by measuring the memory costs when n = 10^5, 10^6 and 10^7 and ID bit length = 32.

For SORT, firstly obtain the SORT fanout data by executing ``optimizer`` and input n, bit length and #layers (which should be lglg(32) = 5). The fanout data will be written to ``settings.txt``. Now, execute ``build/test_trie`` and input n, bit length, #layers, and your fanout array (input 5 numbers, separated by a space) in ``settings.txt``. The program will generate n random integers, insert them into the radix tree, and output the memory statistics.

For vEB-tree, execute ``build/test_trie`` again. The number of layers should be also 5, but the fanout array should be fixed as {16, 8, 4, 2, 2}.

For Figures 12(a), (c), (d), simply execute ``build/test_transform_continuous`` and you will see how the fanout changes when n grows from 10^5 to 10^7.

We did not provide a specific script to plot Figure 12(a), (b), (d) but you can see the corresponding data from the output of the programs. To plot Figure 12(c), after the above steps, execute the following command.

In [ ]:
%%bash
python3 plot_memory_log.py

In [ ]:
from IPython.display import IFrame, display

display(IFrame("figures/memory_log.pdf", width=400, height=300))

#### Figure 13

Execute ``build/test_trie_workload`` and input n = 10^7, bit length = 32. It will ask you which workload you want to evaluate. Run each workload for once (which means, you need to execute the program for 3 times). After that, execute the following command to plot Figure 13.

In [ ]:
%%bash
python3 plot_workload_log.py

In [ ]:
from IPython.display import IFrame, display

display(IFrame("figures/workload_log.pdf", width=800, height=300))

### Comparing SORT and ART (Tables 5, 6)

#### Prepare ART and compile RadixGraph with ART

In [ ]:
%%bash
git clone https://github.com/laurynas-biveinis/unodb.git
cd unodb
git submodule update --init --recursive
cd ..
rm -rf build
mkdir build
cd build
cmake .. -DSTATS=OFF

Enable SORT and ART by setting ``#define USE_SORT 1`` and ``#define USE_ART 1`` in ``src/radixgraph.h``.

In [ ]:
%%bash
sed -i \
  -e 's/^#define[[:space:]]\+USE_SORT[[:space:]]\+[01]/#define USE_SORT 1/' \
  -e 's/^#define[[:space:]]\+USE_ART[[:space:]]\+[01]/#define USE_ART 1/' \
  src/radixgraph.h

Then recompile RadixGraph.

In [ ]:
%%bash
cd build
make -j

Execute ``build/test_vertex_index`` with different n and bit length. Then you can derive the results in Table 5.

For Table 6, when you want to evaluate SORT-based RadixGraph, set ``USE_SORT`` as 1 and ``USE_ART`` as 0; if you want to evaluate ART-based RadixGraph, set ``USE_SORT`` as 0 and ``USE_ART`` as 1. Each time you reset, recompile RadixGraph by following commands.

In [ ]:
%%bash
cd build
make clean && make -j

After you compile, execute ``build/test_radixgraph <edge_file>`` to evaluate RadixGraph on a specific dataset.

Similarly, if you want to disable edge chain in RadixGraph, set ``USE_EDGE_CHAIN`` as 0 in ``source/radixgraph.h`` and recompile. Then execute ``build/test_analytics <edge_file>`` to evaluate RadixGraph for graph analytics.

### Batch updates (Table 7)

Simply run ``build/test_batch_updates <edge_file>`` and you can derive the results. For other baselines, follow their procedures to reproduce. One thing you need to take care is that they evaluate their memory consumptions by theoretical estimation. You need to measure end-to-end memory by yourself.